In [ ]:
!pip install ydata-synthetic==1.1.0


In [ ]:
from ydata_synthetic.synthesizers.regular import RegularSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#Load data and define the data processor parameters
df = pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/mobile-churn.csv')
#filling missing value with mean
df = df.fillna(df.mean())
df=df.drop(['year','month','user_account_id'],axis=1)

df_numerical_features = df.select_dtypes(exclude='object')
df_categorical_features = df.select_dtypes(include='object')

num_cols = list(df_numerical_features)
cat_cols = list(df_categorical_features)


In [ ]:
train_data = df.loc[ df['churn']==1 ].copy()

train_data

,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,...,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage,churn
3,1013,0,11,5.62,0.00,1,0,0,0,1276,...,0.00,0.0,0.0,0.0,0,0,0,0,0.0,1
11,769,0,90,30.01,0.00,0,0,0,0,90,...,0.00,0.0,0.0,0.0,0,0,0,0,0.0,1
12,732,0,28,2.42,3.68,1,1,0,0,30,...,15.48,0.0,1.0,0.0,26,0,1,0,0.0,1
20,15885,0,1276,0.00,0.00,0,0,0,0,1276,...,0.00,0.0,0.0,0.0,0,0,0,0,0.0,1
50,15885,0,8,0.00,0.00,1,0,0,0,1276,...,0.00,0.0,0.0,0.0,0,0,0,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66461,15947,0,1,0.00,0.00,0,0,0,1,1,...,0.00,0.0,0.0,0.0,0,0,0,0,0.0,1
66462,15947,0,1,0.00,0.00,0,0,0,1,1,...,0.00,0.0,0.0,0.0,0,0,0,0,0.0,1
66463,15947,0,1,0.00,0.00,0,0,0,1,1,...,0.00,0.0,0.0,0.0,0,0,0,0,0.0,1
66464,15947,0,1,0.00,0.00,0,0,0,1,1,...,0.00,0.0,0.0,0.0,0,0,0,0,0.0,1


In [ ]:


#Defining the training parameters
noise_dim = 128
dim = 128
batch_size = 500

log_step = 100
epochs = 100
learning_rate = [5e-4, 3e-3]
beta_1 = 0.5
beta_2 = 0.9
models_dir = '../cache'

gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           betas=(beta_1, beta_2),
                           noise_dim=noise_dim,
                           layers_dim=dim)

train_args = TrainParameters(epochs=epochs,
                             sample_interval=log_step)

synth = RegularSynthesizer(modelname='wgangp', model_parameters=gan_args, n_critic=2)
synth.fit(train_data, train_args, num_cols, cat_cols)

synth.save('adult_wgangp_model.pkl')



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
  1%|          | 1/100 [00:02<04:22,  2.65s/it]

Epoch: 0 | disc_loss: 0.007400689646601677 | gen_loss: -0.009581437334418297


  2%|▏         | 2/100 [00:04<03:04,  1.88s/it]

Epoch: 1 | disc_loss: 0.37692388892173767 | gen_loss: -0.024539796635508537


  3%|▎         | 3/100 [00:05<02:40,  1.65s/it]

...

Epoch: 97 | disc_loss: -0.006056288722902536 | gen_loss: -0.05029260367155075


 99%|█████████▉| 99/100 [01:54<00:01,  1.18s/it]

Epoch: 98 | disc_loss: 0.0006320532411336899 | gen_loss: -0.05864463001489639


100%|██████████| 100/100 [01:55<00:00,  1.16s/it]

Epoch: 99 | disc_loss: 0.143892303109169 | gen_loss: -0.060547128319740295


In [ ]:
#########################################################
#    Loading and sampling from a trained synthesizer    #
#########################################################
synth = RegularSynthesizer.load('adult_wgangp_model.pkl')
synth_data = synth.sample(38000)

Synthetic data generation: 100%|██████████| 77/77 [00:00<00:00, 171.30it/s]


In [ ]:
synth_data[synth_data['churn']==1]

,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,...,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage,churn
0,2278,0,73,12.610874,23.945307,0,0,0,0,-63,...,4.027081,16.751657,97.857880,-23.332878,677,-42,786,68,180.824783,1
8,4723,0,76,18.527948,13.796811,0,0,0,0,296,...,73.571487,4.992220,10.408254,-1.679820,564,-14,447,52,120.190880,1
9,3930,0,134,12.884452,3.796242,0,0,0,0,595,...,37.990417,2.818837,-77.503517,-2.134897,323,-6,374,49,63.285828,1
11,3505,0,139,12.147696,5.015007,0,0,0,0,672,...,17.438700,3.250526,-72.462288,-4.424931,314,-2,310,40,76.459389,1
12,4416,0,51,16.022165,15.209270,0,0,0,0,-115,...,9.306839,7.931218,-5.682968,-11.107733,650,-35,588,60,80.953056,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38488,4456,0,58,18.944763,4.346639,0,0,0,0,-59,...,-44.293045,5.533435,-19.187006,-5.606679,580,-18,570,51,48.976112,1
38494,4634,0,66,20.017630,0.358034,0,0,0,0,-54,...,-61.085407,3.937829,-32.618736,-9.575832,513,-15,512,49,39.439640,1
38495,489,0,90,6.436095,32.810947,0,0,0,0,45,...,24.504187,25.255531,214.101288,-40.230095,803,-72,980,91,280.426636,1
38497,3757,0,70,17.746288,22.234541,0,0,0,0,-35,...,20.738281,11.412079,43.335732,-9.942945,690,-37,744,64,149.604279,1


In [ ]:
#concat original data and gan data
data_concat = pd.concat([df, synth_data])
# combine data churn and not churn
data=pd.concat([df, data_concat])

In [ ]:
data

,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,...,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage,churn
0,1000,0,1,0.050000,0.000000,1,1,0,0,66,...,75.270000,0.000000,63.430000,0.000000,210,1,84,0,0.000000,0
1,1000,0,25,28.310000,3.450000,1,0,0,0,1276,...,13.380000,11.180000,2.000000,11.180000,0,0,0,0,0.000000,0
2,1005,0,8,15.620000,1.970000,1,0,0,0,1276,...,30.000000,0.000000,0.000000,10.450000,0,0,0,0,0.000000,0
3,1013,0,11,5.620000,0.000000,1,0,0,0,1276,...,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,1
4,1032,0,2,5.860000,0.150000,1,0,0,1,17,...,2.580000,0.000000,1.000000,0.000000,0,0,0,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38495,489,0,90,6.436095,32.810947,0,0,0,0,45,...,24.504187,25.255531,214.101288,-40.230095,803,-72,980,91,280.426636,1
38496,4834,0,118,2.566816,6.960186,0,0,0,0,1045,...,-128.948242,-2.409219,-93.170837,-3.159625,3,-33,-124,7,51.214771,0
38497,3757,0,70,17.746288,22.234541,0,0,0,0,-35,...,20.738281,11.412079,43.335732,-9.942945,690,-37,744,64,149.604279,1
38498,3601,0,157,-0.017275,11.687100,0,0,0,0,1088,...,-77.785606,-2.787122,-154.670273,1.085711,53,-36,-148,6,52.881634,0


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from numpy import mean
from numpy import std
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
import time

start1=time.time()

X=data.drop(['churn'],axis=1)
y=data["churn"]

enn = EditedNearestNeighbours(n_neighbors=3)
X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
end1 = time.time()
print("The time of execution of preprocess:",
      (end1-start1), "s")

The time of execution of preprocess: 46.58066439628601 s


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-tn-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangpenn-tn-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.969895 using {'class_weight': {0: 100, 1: 60}}
0.969097 (0.001064) with: {'class_weight': {0: 100, 1: 10}}
0.968585 (0.001614) with: {'class_weight': {0: 100, 1: 20}}
0.968820 (0.001512) with: {'class_weight': {0: 100, 1: 30}}
0.969065 (0.001397) with: {'class_weight': {0: 100, 1: 40}}
0.969371 (0.001454) with: {'class_weight': {0: 100, 1: 50}}
0.969895 (0.001447) with: {'class_weight': {0: 100, 1: 60}}
0.969333 (0.001318) with: {'class_weight': {0: 100, 1: 70}}
0.969393 (0.001428) with: {'class_weight': {0: 100, 1: 80}}
0.969384 (0.001244) with: {'class_weight': {0: 100, 1: 90}}
0.969226 (0.001118) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-tn-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-tn-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.967309 using {'class_weight': {0: 100, 1: 60}}
0.965533 (0.001350) with: {'class_weight': {0: 100, 1: 10}}
0.965457 (0.001604) with: {'class_weight': {0: 100, 1: 20}}
0.966139 (0.001141) with: {'class_weight': {0: 100, 1: 30}}
0.966908 (0.001457) with: {'class_weight': {0: 100, 1: 40}}
0.966729 (0.001153) with: {'class_weight': {0: 100, 1: 50}}
0.967309 (0.001420) with: {'class_weight': {0: 100, 1: 60}}
0.966841 (0.001445) with: {'class_weight': {0: 100, 1: 70}}
0.966924 (0.001361) with: {'class_weight': {0: 100, 1: 80}}
0.967113 (0.001421) with: {'class_weight': {0: 100, 1: 90}}
0.967176 (0.001124) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-tn-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-tn-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.967648 using {'class_weight': {0: 100, 1: 60}}
0.966481 (0.001005) with: {'class_weight': {0: 100, 1: 10}}
0.966221 (0.001465) with: {'class_weight': {0: 100, 1: 20}}
0.966369 (0.001387) with: {'class_weight': {0: 100, 1: 30}}
0.967046 (0.001192) with: {'class_weight': {0: 100, 1: 40}}
0.967447 (0.001295) with: {'class_weight': {0: 100, 1: 50}}
0.967648 (0.001465) with: {'class_weight': {0: 100, 1: 60}}
0.967345 (0.001389) with: {'class_weight': {0: 100, 1: 70}}
0.967406 (0.001251) with: {'class_weight': {0: 100, 1: 80}}
0.967440 (0.001119) with: {'class_weight': {0: 100, 1: 90}}
0.967346 (0.001171) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-tn-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-tn-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.933638 using {'class_weight': {0: 100, 1: 90}}
0.929286 (0.002017) with: {'class_weight': {0: 100, 1: 10}}
0.932246 (0.001987) with: {'class_weight': {0: 100, 1: 20}}
0.932284 (0.001822) with: {'class_weight': {0: 100, 1: 30}}
0.933099 (0.001946) with: {'class_weight': {0: 100, 1: 40}}
0.933060 (0.001988) with: {'class_weight': {0: 100, 1: 50}}
0.932363 (0.001911) with: {'class_weight': {0: 100, 1: 60}}
0.932277 (0.002096) with: {'class_weight': {0: 100, 1: 70}}
0.932215 (0.001866) with: {'class_weight': {0: 100, 1: 80}}
0.933638 (0.001554) with: {'class_weight': {0: 100, 1: 90}}
0.933026 (0.002124) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-tn-lr-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-tn-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.877824 using {'class_weight': {0: 100, 1: 90}}
0.780289 (0.016688) with: {'class_weight': {0: 100, 1: 10}}
0.815177 (0.005985) with: {'class_weight': {0: 100, 1: 20}}
0.838607 (0.008703) with: {'class_weight': {0: 100, 1: 30}}
0.864857 (0.005334) with: {'class_weight': {0: 100, 1: 40}}
0.872042 (0.003664) with: {'class_weight': {0: 100, 1: 50}}
0.873126 (0.003494) with: {'class_weight': {0: 100, 1: 60}}
0.873763 (0.003174) with: {'class_weight': {0: 100, 1: 70}}
0.874509 (0.003271) with: {'class_weight': {0: 100, 1: 80}}
0.877824 (0.002024) with: {'class_weight': {0: 100, 1: 90}}
0.874322 (0.003023) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-tn-lr-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-tn-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/mobile"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.853316 using {'class_weight': {0: 100, 1: 90}}
0.697988 (0.023263) with: {'class_weight': {0: 100, 1: 10}}
0.747757 (0.008260) with: {'class_weight': {0: 100, 1: 20}}
0.783798 (0.012429) with: {'class_weight': {0: 100, 1: 30}}
0.824665 (0.007119) with: {'class_weight': {0: 100, 1: 40}}
0.837325 (0.004850) with: {'class_weight': {0: 100, 1: 50}}
0.842657 (0.004069) with: {'class_weight': {0: 100, 1: 60}}
0.846454 (0.003944) with: {'class_weight': {0: 100, 1: 70}}
0.848865 (0.004037) with: {'class_weight': {0: 100, 1: 80}}
0.853316 (0.003054) with: {'class_weight': {0: 100, 1: 90}}
0.852658 (0.002921) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
#balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
#balance1=[{0:10,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-mobile-svm-cs-auc.xlsx', index=False)



/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best: 0.937270 using {'class_weight': {0: 40, 1: 100}}
0.923806 (0.001288) with: {'class_weight': {0: 10, 1: 100}}
0.933926 (0.000956) with: {'class_weight': {0: 20, 1: 100}}
0.936749 (0.001259) with: {'class_weight': {0: 30, 1: 100}}
0.937270 (0.000615) with: {'class_weight': {0: 40, 1: 100}}
0.936062 (0.000702) with: {'class_weight': {0: 50, 1: 100}}
0.936540 (0.000887) with: {'class_weight': {0: 60, 1: 100}}
0.934224 (0.001269) with: {'class_weight': {0: 70, 1: 100}}
0.932668 (0.000672) with: {'class_weight': {0: 80, 1: 100}}
0.932293 (0.000807) with: {'class_weight': {0: 90, 1: 100}}
0.931695 (0.000766) with: {'class_weight': {0: 100, 1: 100}}


ValueError: mount failed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the source file path (in Colab)
source_file = '/content/wgangp-mobile-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/"

# Copy the file to the destination
!cp $source_file $destination_folder

Mounted at /content/drive


In [ ]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-mobile-svm-cs-f1.xlsx', index=False)

from google.colab import drive
drive.mount('/content/drive')

# Define the source file path (in Colab)
source_file = '/content/wgangp-mobile-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/"

# Copy the file to the destination
!cp $source_file $destination_folder

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best: 0.869268 using {'class_weight': {0: 100, 1: 100}}
0.770446 (0.001722) with: {'class_weight': {0: 10, 1: 100}}
0.835189 (0.001959) with: {'class_weight': {0: 20, 1: 100}}
0.847756 (0.002299) with: {'class_weight': {0: 30, 1: 100}}
0.858912 (0.001839) with: {'class_weight': {0: 40, 1: 100}}
0.863797 (0.001447) with: {'class_weight': {0: 50, 1: 100}}
0.869203 (0.001700) with: {'class_weight': {0: 60, 1: 100}}
0.867293 (0.001518) with: {'class_weight': {0: 70, 1: 100}}
0.868007 (0.001555) with: {'class_weight': {0: 80, 1: 100}}
0.868653 (0.001462) with: {'class_weight': {0: 90, 1: 100}}
0.869268 (0.001060) with: {'class_weight': {0: 100, 1: 100}}
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from imblearn.metrics import geometric_mean_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
#balance1=[{0:10,1:100}]
param_grid = dict(class_weight=balance)
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-tn-mobile-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangp-tn-mobile-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/"

# Copy the file to the destination
!cp $source_file $destination_folder

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best: 0.880958 using {'class_weight': {0: 100, 1: 100}}
0.757924 (0.002906) with: {'class_weight': {0: 100, 1: 10}}
0.846384 (0.001603) with: {'class_weight': {0: 100, 1: 20}}
0.846808 (0.001578) with: {'class_weight': {0: 100, 1: 30}}
0.852958 (0.001511) with: {'class_weight': {0: 100, 1: 40}}
0.853824 (0.001619) with: {'class_weight': {0: 100, 1: 50}}
0.854285 (0.001419) with: {'class_weight': {0: 100, 1: 60}}
0.855173 (0.001873) with: {'class_weight': {0: 100, 1: 70}}
0.878465 (0.001782) with: {'class_weight': {0: 100, 1: 80}}
0.878540 (0.001713) with: {'class_weight': {0: 100, 1: 90}}
0.880958 (0.002847) with: {'class_weight': {0: 100, 1: 100}}
